<a href="https://colab.research.google.com/github/unt-iialab/INFO5731_Spring2020/blob/master/Assignments/INFO5731_Assignment_Two.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **INFO5731 Assignment Two**

In this assignment, you will try to gather text data from open data source via web scraping or API. After that you need to clean the text data and syntactic analysis of the data.

# **Question 1**

(40 points). Write a python program to collect text data from **either of the following sources** and save the data into a **csv file**:

(1) Collect all the customer reviews of the product [Apple iPhone 11](https://www.amazon.com/Apple-iPhone-11-64GB-Unlocked/dp/B07ZPKF8RG/ref=sr_1_13?dchild=1&keywords=iphone+12&qid=1631721363&sr=8-13) on amazon.

(2) Collect the top 10000 User Reviews of the film [Shang-Chi and the Legend of the Ten Rings](https://www.imdb.com/title/tt9376612/reviews?ref_=tt_sa_3) from IMDB.

(3) Collect all the reviews of the top 100 most popular software from [G2](https://www.g2.com/) or [Capterra](https://www.capterra.com/)

(4) Collect the abstracts of the top 10000 research papers by using the query [natural language processing](https://citeseerx.ist.psu.edu/search?q=natural+language+processing&submit.x=0&submit.y=0&sort=rlv&t=doc) from CiteSeerX.

(5) Collect all the information of the 904 narrators in the [Densho Digital Repository](https://ddr.densho.org/narrators/).

(6) Collect the top 10000 tweets by using hashtag ["#blacklivesmatter"](https://twitter.com/hashtag/blacklivesmatter) from Twitter. 


In [1]:
# Write your code here

!pip install selenium
!pip install beautifulsoup4
!apt-get update # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
import sys, time
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
import selenium
#webdriver is our tool to interact with the webpage
from selenium import webdriver
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')


import requests #needed to load the page for BS4
from bs4 import BeautifulSoup
import pandas as pd #Using panda to create our dataframe

Hit:1 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:4 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:5 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:6 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:7 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Hit:8 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Get:9 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:11 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:12 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:13 https://developer.download.nvidia.com/compute/machine-le

/usr/local/lib/python3.7/dist-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.8) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [2]:
wd = webdriver.Chrome('chromedriver',chrome_options=chrome_options)
wd.get("https://www.imdb.com/title/tt9376612/reviews")
try:
  count = 10000
  while(count>0):
    wd.find_element_by_id("load-more-trigger").click()
    wd.execute_script('window.scrollTo(0,document.body.scrollHeight);')
    time.sleep(1)
    new_height = wd.execute_script('return document.body.scrollHeight')
    prev_height = new_height
    count-=1
except Exception:
  pass

page = wd.page_source
soup = BeautifulSoup(page, "html.parser")

#Get the title of the movie
all = soup.find(id="main")
parent = all.find(class_ ="parent")
name = parent.find(itemprop = "name")
url = name.find(itemprop = 'url')
film_title = url.get_text()

#Get the title of the review
title = [t.get_text().replace("\n", "") for t in all.select(".title")]

#Get the user name of the review
username = [n.find('a').text for n in all.select(".display-name-date .display-name-link")]

#Get the review
review = [r.get_text() for r in all.select(".content .text")]

#Get the review data
date = [d.get_text() for d in all.select(".display-name-date .review-date")]

#Make it into dataframe
table_review = pd.DataFrame({
  "UserName": username,
  "Title" : title,
  "Review" : review,
  "Date": date
})

table_review

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: use options instead of chrome_options
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  


,UserName,Title,Review,Date
0,MartinHafer,A visual feast.,"I am not a big fan of the many, many Marvel fi...",1 December 2021
1,boblipton,Next Phase,It was an okay movie. Simu Liu is the son of t...,11 September 2021
2,nogodnomasters,Bus Boy,Shaun (Simu Liu) and Katy (Awkwafina) are park...,11 September 2021
3,Sleepin_Dragon,Wow!,"I was not expecting that, I had visions of a f...",9 September 2021
4,kosmasp,Precious ... ten times,"Of course I am just riffing off, teasing and m...",12 September 2021
...,...,...,...,...
2351,gullymark,Phase 4 in the MCU has a bright future,My review..\nShang-Chi and the Legend of the T...,5 September 2021
2352,thephenomanish,A masterpiece,The best solo marvel movie if not the best mov...,11 November 2021
2353,abansal-40967,This is a cinematic marvel,There are no words to describe the look of thi...,8 September 2021
2354,owenlong-52387,Tony Leung upgrades it to another level! Acti...,I am not interested in popcorn movies at all. ...,6 September 2021


# **Question 2**

(30 points). Write a python program to **clean the text data** you collected above and save the data in a new column in the csv file. The data cleaning steps include:

(1) Remove noise, such as special characters and punctuations.

(2) Remove numbers.

(3) Remove stopwords by using the [stopwords list](https://gist.github.com/sebleier/554280).

(4) Lowercase all texts

(5) Stemming. 

(6) Lemmatization.

In [3]:
# Write your code here

#before cleaning
print("Before Cleaning.\n", table_review.Review.head(), end="\n\n")

#1
table_review.Review = table_review.Review.str.replace('[^\w\s]','')
print("(1) Remove noise, such as special characters and punctuations.\n", table_review.Review.head(), end="\n\n")

#2
table_review.Review = table_review.Review.str.replace('\d+','')
print("(2) Remove numbers.\n", table_review.Review.head(), end="\n\n")

#3
import requests
response = requests.get("https://gist.githubusercontent.com/sebleier/554280/raw/7e0e4a1ce04c2bb7bd41089c9821dbcf6d0c786c/NLTK's%2520list%2520of%2520english%2520stopwords")
stopwords = [word for word in response.text.replace('\n', ' ').split()]
table_review.Review = table_review.Review.apply(lambda sent: " ".join((word for word in sent.split() if word.lower() not in stopwords)))
print("(3) Remove stopwords by using the stopwords list.\n", table_review.Review.head(), end="\n\n")

#4
table_review.Review = table_review.Review.apply(lambda sent: sent.lower())
print("(4) Lowercase all texts.\n", table_review.Review.head(), end="\n\n")

#5
from nltk.stem import PorterStemmer
stemmer = PorterStemmer()
table_review.Review.apply(lambda sent: " ".join([stemmer.stem(word) for word in sent.split()]))
print("(5) Stemming.\n", table_review.Review.head(), end="\n\n")

#6
from textblob import Word
import nltk
nltk.download('wordnet')
table_review.Review = table_review.Review.apply(lambda x: " ".join([Word(word).lemmatize() for word in x.split()]))
print("(6) Lemmatization.\n", table_review.Review.head(), end="\n\n")

Before Cleaning.
 0    I am not a big fan of the many, many Marvel fi...
1    It was an okay movie. Simu Liu is the son of t...
2    Shaun (Simu Liu) and Katy (Awkwafina) are park...
3    I was not expecting that, I had visions of a f...
4    Of course I am just riffing off, teasing and m...
Name: Review, dtype: object

(1) Remove noise, such as special characters and punctuations.
 0    I am not a big fan of the many many Marvel fil...
1    It was an okay movie Simu Liu is the son of th...
2    Shaun Simu Liu and Katy Awkwafina are parking ...
3    I was not expecting that I had visions of a fi...
4    Of course I am just riffing off teasing and ma...
Name: Review, dtype: object

(2) Remove numbers.
 0    I am not a big fan of the many many Marvel fil...
1    It was an okay movie Simu Liu is the son of th...
2    Shaun Simu Liu and Katy Awkwafina are parking ...
3    I was not expecting that I had visions of a fi...
4    Of course I am just riffing off teasing and ma...
Name: Review, 

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: FutureWarning: The default value of regex will change from True to False in a future version.
  import sys
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: FutureWarning: The default value of regex will change from True to False in a future version.
  # This is added back by InteractiveShellApp.init_path()


(3) Remove stopwords by using the stopwords list.
 0    big fan many many Marvel films reason saw one ...
1    okay movie Simu Liu son seemingly immortal mas...
2    Shaun Simu Liu Katy Awkwafina parking valets u...
3    expecting visions film along lines Snake Eyes ...
4    course riffing teasing making fun certain thin...
Name: Review, dtype: object

(4) Lowercase all texts.
 0    big fan many many marvel films reason saw one ...
1    okay movie simu liu son seemingly immortal mas...
2    shaun simu liu katy awkwafina parking valets u...
3    expecting visions film along lines snake eyes ...
4    course riffing teasing making fun certain thin...
Name: Review, dtype: object

(5) Stemming.
 0    big fan many many marvel films reason saw one ...
1    okay movie simu liu son seemingly immortal mas...
2    shaun simu liu katy awkwafina parking valets u...
3    expecting visions film along lines snake eyes ...
4    course riffing teasing making fun certain thin...
Name: Review, dtype: obje

# **Question 3**

(30 points). Write a python program to conduct **syntax and structure analysis** of the clean text you just saved above. The syntax and structure analysis includes: 

(1) Parts of Speech (POS) Tagging: Tag Parts of Speech of each word in the text, and calculate the total number of N(oun), V(erb), Adj(ective), Adv(erb), respectively.

(2) Constituency Parsing and Dependency Parsing: print out the constituency parsing trees and dependency parsing trees of all the sentences. Using one sentence as an example to explain your understanding about the constituency parsing tree and dependency parsing tree.

(3) Named Entity Recognition: Extract all the entities such as person names, organizations, locations, product names, and date from the clean texts, calculate the count of each entity.

In [5]:
# Write your code here


#1
import spacy
nltk.download('punkt')
from spacy import displacy
from nltk import word_tokenize, pos_tag, pos_tag_sents
nltk.download('averaged_perceptron_tagger')
from nltk.tokenize import word_tokenize
from collections import Counter
#Tag Parts of Speech of each word in the text
pos = [nltk.pos_tag(word_tokenize(sent)) for sent in table_review.Review]
print(pos, end="\n\n")
#calculate the total number of N(oun), V(erb), Adj(ective), Adv(erb)
count = [Counter(tag for words, tag in tags) for tags in pos]
print(count, end="\n\n")

#2
# !cd ~
# !wget http://nlp.stanford.edu/software/stanford-corenlp-full-2018-02-27.zip
# !unzip stanford-corenlp-full-2018-02-27.zip
# !cd stanford-corenlp-full-2018-02-27
# !java -Xmx4g -cp "*" edu.stanford.nlp.pipeline.StanfordCoreNLPServer \
# -serverProperties StanfordCoreNLP-chinese.properties \
# -preload tokenize,ssplit,pos,lemma,ner,parse \
# -status_port 9001  -port 9001 -timeout 15000
# from nltk.parse.corenlp import CoreNLPDependencyParser
# dep_parser = CoreNLPDependencyParser(url='http://localhost:9001')
# sent = "I shot an elephant with a banana .".split()
# parses = list(dep_parser.parse(sent))
# parses[0].tree().pretty_print()
import nltk
from nltk import Production, CFG
# grammar
cgrammar = nltk.CFG.fromstring("""
S -> NP VP
VP -> V NP
PP -> P NP
NP -> NP PP | Det N | 'Peter' | 'Denver'
V -> 'prefers'
P -> 'from'
N -> 'flight'
Det -> 'the'
""")
# print grammar
print(cgrammar, end='\n')
sent = ['Peter', 'prefers', 'the', 'flight', 'from', 'Denver']
# Using Chart Parser
cparser = nltk.ChartParser(cgrammar)
for tree in cparser.parse(sent):
  print(tree, end="\n")
# drawing trees
!pip install svgling==0.3.1
import svgling
svgling.draw_tree(tree)
print("\n")


#3
import en_core_web_sm
nlp = en_core_web_sm.load()
nep = list()
for sentence in table_review.Review:
  doc=nlp(sentence)
  for X in doc.ents:
    if X.text and X.label_:
      nep.append((X.text, X.label_))
print(nep, end="\n")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[[('big', 'JJ'), ('fan', 'NN'), ('many', 'JJ'), ('many', 'JJ'), ('marvel', 'NN'), ('film', 'NN'), ('reason', 'NN'), ('saw', 'VBD'), ('one', 'CD'), ('oldest', 'JJS'), ('daughter', 'NN'), ('insisted', 'VBD'), ('watch', 'NN'), ('itand', 'NN'), ('overall', 'JJ'), ('impressed', 'JJ'), ('movie', 'NN'), ('though', 'IN'), ('think', 'NN'), ('enjoyable', 'JJ'), ('insane', 'NN'), ('eye', 'NN'), ('candy', 'NN'), ('story', 'NN'), ('itselfi', 'NN'), ('could', 'MD'), ('talk', 'VB'), ('plot', 'NN'), ('noticed', 'VBN'), ('review', 'NN'), ('film', 'NN'), ('ill', 'NNS'), ('say', 'VBP'), ('looked', 'JJ'), ('great', 'JJ'), ('would', 'MD'), ('great', 'JJ'), ('seen', 'VBN'), ('big', 'JJ'), ('screen', 'JJ'), ('story

**Write your explanations of the constituency parsing tree and dependency parsing tree here (Question 3-2):** 

Parsing is the task of generating a parse tree from a given sentence in computational linguistics. According to formal grammar, a parse tree illustrates the syntactical structure of a sentence. Constituency and dependency parsing are two separate sorts of grammar-based techniques. According to a context-free grammar, a constituency parse tree comprises a syntactical representation of a phrase. The sentences are divided into single phrasal elements in this representation, which is very hierarchical. The syntax of a sentence is expressed in terms of directed edges between words in a dependency parse tree.